In [11]:
import bz2
import os
import numpy as np

In [3]:
from keras.models import Sequential
from keras.layers import Dense

In [4]:
model = Sequential()

In [5]:
model.add(Dense(units=64, activation='relu', input_dim=100))
model.add(Dense(units=10, activation='softmax'))

In [6]:
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [12]:
# Load embeddings
embedding_index = {}
f = bz2.open("/Users/serhiinechyporhuk/courses/ubercorpus.cased.tokenized.glove.300d.bz2")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embedding_index[word] = coefs
f.close()

In [ ]:
print("Number of word embeddings:", )